In [34]:
import numpy as np
from numpy import asarray
import os
import cv2
import pandas as pd

In [35]:
# data paths
train_data_dir = r'C:\Users\prath\Downloads\Machine_Learning\Dogs\train'
test_data_dir = r'C:\Users\prath\Downloads\Machine_Learning\Dogs\test'
val_data_dir = r'C:\Users\prath\Downloads\Machine_Learning\Dogs\valid'
csv_path = r'C:\Users\prath\Downloads\Machine_Learning\Dogs\dogs.csv'

In [36]:
# create image path array for above dirs
def create_image_path_array(path):
    folder_paths = [path+'\\'+each for each in os.listdir(path)]
#     print(len(folder_paths))
    classes = [each for each in os.listdir(path)]
#     print(classes)
    image_paths = []
    for folder in folder_paths:
        image_paths = image_paths + [folder+'\\'+img for img in os.listdir(folder)]
    return image_paths

In [37]:
# create separate image path arrays for train, test, valid
train_image_paths = create_image_path_array(train_data_dir)
test_image_paths = create_image_path_array(test_data_dir)
val_image_paths = create_image_path_array(val_data_dir)

In [38]:
# one hot encoding given csv file with labels and test,train,valid marked
def one_hot_encoding(csv_path):
    df = pd.read_csv(csv_path)
    one_hot = pd.get_dummies(df.labels)
    class_names = pd.unique(df['labels'])
    print(class_names)
    df = df.drop('filepaths', axis = 1).drop('labels', axis = 1).join(one_hot)
    ytrain = df[df['data set'] == 'train'].drop('data set', axis = 1).to_numpy()
    ytest = df[df['data set'] == 'test'].drop('data set', axis = 1).to_numpy()
    yval = df[df['data set'] == 'valid'].drop('data set', axis = 1).to_numpy()
    return ytrain, ytest, yval, class_names

In [46]:
ytrain, ytest, yval, class_names = one_hot_encoding(csv_path)
ytrain = np.vstack((ytrain, yval))
df = pd.DataFrame(class_names)
# df.to_csv(r'C:\Users\prath\Downloads\Machine_Learning\Dogs\classification_report.csv', mode='a', index=False, header=False)

['Afghan' 'African Wild Dog' 'Airedale' 'American Hairless'
 'American Spaniel' 'Basenji' 'Basset' 'Beagle' 'Bearded Collie'
 'Bermaise' 'Bichon Frise' 'Blenheim' 'Bloodhound' 'Bluetick'
 'Border Collie' 'Borzoi' 'Boston Terrier' 'Boxer' 'Bull Mastiff'
 'Bull Terrier' 'Bulldog' 'Cairn' 'Chihuahua' 'Chinese Crested' 'Chow'
 'Clumber' 'Cockapoo' 'Cocker' 'Collie' 'Corgi' 'Coyote' 'Dalmation'
 'Dhole' 'Dingo' 'Doberman' 'Elk Hound' 'French Bulldog' 'German Sheperd'
 'Golden Retriever' 'Great Dane' 'Great Perenees' 'Greyhound'
 'Groenendael' 'Irish Spaniel' 'Irish Wolfhound' 'Japanese Spaniel'
 'Komondor' 'Labradoodle' 'Labrador' 'Lhasa' 'Malinois' 'Maltese'
 'Mex Hairless' 'Newfoundland' 'Pekinese' 'Pit Bull' 'Pomeranian' 'Poodle'
 'Pug' 'Rhodesian' 'Rottweiler' 'Saint Bernard' 'Schnauzer'
 'Scotch Terrier' 'Shar_Pei' 'Shiba Inu' 'Shih-Tzu' 'Siberian Husky'
 'Vizsla' 'Yorkie']


In [40]:
def image_processing(image_path_array, y):
    x = []
    yout = []
    for i,image in enumerate(image_path_array):
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        scale_down = 0.4
        img = cv2.resize(img, None, fx= scale_down, fy= scale_down, interpolation= cv2.INTER_LINEAR)
        x.append(img)
#         randomize augmentations(maybe use another loop outside this one)
#         add augmentations here
#         create 1 hot labels for them
    return x,yout

In [41]:
xtrain, ytrain = np.array(image_processing(train_image_paths, ytrain))
xtest,ytest = np.array(image_processing(test_image_paths, ytest)).astype('uint8')
xval,yval = np.array(image_processing(val_image_paths, yval))
xtrain = np.vstack((xtrain, xval)).astype('uint8')
np.shape(xtrain)

(8646, 90, 90, 3)

In [42]:
# saving this as an npz file just in case image processing takes too long to run
np.savez_compressed('Dogs_npz', xtrain, ytrain, xtest, ytest)